# DS776 Course Setup

Welcome to DS776 Deep Learning! This notebook will set up your course environment.

**This notebook will:**
1. Install the `introdl` course package
2. Create your workspace folders
3. Set up API key configuration
4. Verify everything is working

## ⚠️ Important Notes:
- Run this notebook ONCE at the beginning of the course
- Run all cells in order
- The setup takes about 1-2 minutes

## Step 1: Install Course Package

This installs the `introdl` package which contains all course utilities and helper functions.

In [ ]:
# Install the introdl package
import subprocess
import sys
from pathlib import Path

print("📦 Installing introdl course package...")
print("This may take 1-2 minutes...\n")

# Get the path to the introdl package
course_tools_dir = Path.cwd().parent  # Assuming we're in Course_Tools
introdl_dir = course_tools_dir / "introdl"

if not introdl_dir.exists():
    # Try alternative path
    introdl_dir = Path.cwd() / "introdl"

if introdl_dir.exists():
    # Install the package
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-e", str(introdl_dir), "--quiet"],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("✅ Package installed successfully!")
    else:
        print("❌ Installation failed. Error:")
        print(result.stderr)
else:
    print("❌ Could not find introdl package directory")
    print(f"   Looked in: {introdl_dir}")

## Step 2: Create Workspace Structure

Creates the folders needed for storing data, models, and cached files.

In [ ]:
from pathlib import Path
import os

print("📁 Creating workspace folders...\n")

# Determine the course root
current_dir = Path.cwd()
if "Course_Tools" in str(current_dir):
    # We're in Lessons/Course_Tools
    course_root = current_dir.parent.parent
else:
    # Try to find the course root
    for parent in [current_dir] + list(current_dir.parents):
        if (parent / "Lessons").exists() and (parent / "Homework").exists():
            course_root = parent
            break
    else:
        course_root = Path.home() / "DS776"

# Create home_workspace structure
home_workspace = course_root / "home_workspace"
folders_to_create = [
    home_workspace,
    home_workspace / "data",
    home_workspace / "downloads",
    home_workspace / "models"
]

for folder in folders_to_create:
    folder.mkdir(parents=True, exist_ok=True)
    print(f"✅ Created/verified: {folder.relative_to(course_root)}")

# Set environment variable for course root
os.environ['DS776_ROOT_DIR'] = str(course_root)
print(f"\n📍 Course root set to: {course_root}")

## Step 3: Set Up API Keys

Creates an API key configuration file that you can edit with your own keys.

In [ ]:
import shutil

print("🔑 Setting up API key configuration...\n")

# Create api_keys.env in home_workspace if it doesn't exist
api_keys_dest = home_workspace / "api_keys.env"
api_keys_template = course_tools_dir / "api_keys.env"

if not api_keys_dest.exists():
    if api_keys_template.exists():
        # Copy the template
        shutil.copy2(api_keys_template, api_keys_dest)
        print(f"✅ Created API keys file: {api_keys_dest.relative_to(course_root)}")
    else:
        # Create a new template
        template_content = """# API Keys for DS776 Deep Learning Course
# Edit this file with your actual API keys
# This file is in home_workspace and will sync across compute servers

# OpenAI API Key (for GPT models)
OPENAI_API_KEY=your_openai_key_here

# Hugging Face Token (for model downloads)
HF_TOKEN=your_huggingface_token_here

# OpenRouter API Key (for various LLMs)
OPENROUTER_API_KEY=your_openrouter_key_here

# Google API Key (for Gemini models)
GOOGLE_API_KEY=your_google_key_here

# Groq API Key (for fast inference)
GROQ_API_KEY=your_groq_key_here
"""
        api_keys_dest.write_text(template_content)
        print(f"✅ Created API keys template: {api_keys_dest.relative_to(course_root)}")
    
    print("\n📝 TO ADD YOUR API KEYS:")
    print(f"   1. Open {api_keys_dest.relative_to(course_root)}")
    print("   2. Replace 'your_xxx_key_here' with your actual API keys")
    print("   3. Save the file")
    print("\n   Your keys will automatically sync across compute servers!")
else:
    print(f"✅ API keys file already exists: {api_keys_dest.relative_to(course_root)}")
    print("   (Your existing keys have been preserved)")

## Step 4: Verify Installation

Let's make sure everything is working correctly.

In [ ]:
print("🔍 Verifying installation...\n")

# Test 1: Import introdl package
try:
    import introdl
    print("✅ introdl package imported successfully")
    print(f"   Version: {getattr(introdl, '__version__', 'Unknown')}")
except ImportError as e:
    print(f"❌ Could not import introdl: {e}")

# Test 2: Check workspace folders
print("\n📁 Workspace folders:")
for folder in folders_to_create:
    if folder.exists():
        print(f"   ✅ {folder.relative_to(course_root)}")
    else:
        print(f"   ❌ {folder.relative_to(course_root)} - NOT FOUND")

# Test 3: Test config_paths_keys
print("\n🔧 Testing configuration...")
try:
    from introdl.utils import config_paths_keys
    paths = config_paths_keys()
    print("✅ Configuration loaded successfully")
except Exception as e:
    print(f"❌ Configuration error: {e}")

# Test 4: Check PyTorch
print("\n🔥 Checking PyTorch...")
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"   GPU available: {torch.cuda.get_device_name(0)}")
    else:
        print("   Running on CPU (GPU not available)")
except ImportError:
    print("❌ PyTorch not installed")

## 🎉 Setup Complete!

Your DS776 course environment is now configured and ready to use.

### Next Steps:
1. **Add your API keys** to `home_workspace/api_keys.env` if you haven't already
2. **Start with Lesson 01** notebooks
3. **Use Homework utility notebooks** to manage storage in each homework folder

### Need Help?
- **Storage issues?** Use the `Clean_and_Free_Space.ipynb` notebook
- **Package issues?** Use the `Install_Course_Package.ipynb` notebook
- **API key issues?** Check that your keys are correctly entered in `home_workspace/api_keys.env`

Good luck with the course! 🚀